# VLR Scraper

## Main page, scraping match URLs

In [3]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Session with automatic retry on connection errors
session = requests.Session()
retry = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

async def req(url, page):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Finding URLs for all matches
    cards = soup.find_all('div', class_='wf-card')
    match_urls = []

    for card in cards:
        matches = card.find_all('a', class_=['wf-module-item', 'match-item'])
        
        for match in matches:
            match_urls.append('https://www.vlr.gg' + match['href'])

    return match_urls

In [ ]:
match_urls = []

for page in tqdm(range(1, 476), desc="Scraping pages"):
    match_urls += await req(f'https://www.vlr.gg/matches/results?page={page}', page)
    time.sleep(2)

In [5]:
with open('match_urls.json', 'w') as f:
    json.dump(match_urls, f)